[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

[Transfer Learning with VGG-16 CNN+AUG LB 0.1712](https://www.kaggle.com/code/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712)

In [1]:
# This Python 3 enviroment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directiory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))    # 윈도우 사용으로 주석처리

# Any results you write to the current directory are saved as output

TL;DR
**Runs on GPU** There is some compatiblity issue with CPUs

1. Hyperparameters in Deep learning are many, tuning them will take weeks or months. Generally researchers do this tuning and publish paper when they find a nice set of architecture which performs better than other.

2. Since the model is pre-trained, it converges very fast and you but still you need GPU to use this. Due to some library issues, it doesn't work on CPU.

3. For our purpose, we can use those architectures, which are made available by those researchers to us.

4. Using those pretrained nets, layers of which already 'knows' how to extract features, we can don't have to tune the hyperparameters. Since they are already trained of some dataset(say imagenet), their pre-trained weights provide a good initialization of weights and because of this, our Convnet converges very fast which otherwise can take days on these deep architectures. That's the idea behind **Transfer Learning**. Examples of which are VGG16, InceptionNet, goolenet, Resnet etc.

5. In this kernel we will use pretrained VGG-16 network which performs very well on small size images.

**VGG architecture has proved to worked well on small sized images(CIFAR-10)** I expected it to work well for this dataset as well.

1. The code also includes the data augmentation steps, thus considerably improving the performance.

2. **GPU is needed**

Here is the link of the research paper if you are interested. https://arxiv.org/pdf/1409.1556.pdf

Also here is the doc for keras library: https://keras.io/applications/#vgg16

**DeepL 번역**  
TL;DR
**GPU에서 실행** CPU와 일부 호환성 문제가 있습니다.

1. 딥 러닝에는 하이퍼파라미터가 많기 때문에 이를 튜닝하는 데 몇 주 또는 몇 달이 걸립니다. 일반적으로 연구자들은 이러한 튜닝을 통해 다른 아키텍처보다 더 나은 성능을 발휘하는 멋진 아키텍처를 발견하면 논문을 발표합니다.

2. 모델이 사전 학습되어 있기 때문에 매우 빠르게 수렴하지만 이를 사용하려면 여전히 GPU가 필요합니다. 일부 라이브러리 문제로 인해 CPU에서는 작동하지 않습니다.

3. 연구자들이 우리에게 제공한 아키텍처를 사용할 수 있습니다.

4. 이미 특징을 추출하는 방법을 '알고' 있는 사전 훈련된 네트워크를 사용하면 하이퍼파라미터를 조정할 필요가 없습니다. 이미 일부 데이터 세트(예: 이미지넷)에 대해 학습되어 있기 때문에 사전 학습된 가중치는 좋은 가중치 초기화를 제공하며, 이 때문에 이러한 심층 아키텍처에서는 며칠이 걸릴 수 있는 Convnet이 매우 빠르게 수렴합니다. 이것이 바로 **Transfer Learning**의 아이디어입니다. 그 예로 VGG16, InceptionNet, goolenet, Resnet 등이 있습니다.

5. 이 커널에서는 작은 크기의 이미지에서 매우 잘 작동하는 사전 학습된 VGG-16 네트워크를 사용할 것입니다.

**VGG 아키텍처는 작은 크기의 이미지(CIFAR-10)에서 잘 작동하는 것으로 입증되었습니다** 이 데이터 세트에서도 잘 작동할 것으로 예상했습니다.

1. 이 코드에는 데이터 증강 단계도 포함되어 있어 성능이 상당히 향상되었습니다.

2. **GPU가 필요합니다**.

관심이 있으시다면 연구 논문 링크를 참고하세요. https://arxiv.org/pdf/1409.1556.pdf

케라스 라이브러리 관련 문서도 여기 있습니다: https://keras.io/applications/#vgg16

In [1]:
# Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
# plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/train.json")
target_train = train['is_iceberg']
test = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/test.json")

https://www.kaggle.com/code/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712